In [18]:
#MARSS (python)

In [19]:
print("heloo")
!pip install nibabel
!pip install shutil
!pip install pandas
!pip install matplotlib
!pip install scipy
# nipype dependencies
!pip install networkx
!pip install numpy
!pip install traits
!pip install python-dateutil
!pip install Sphinx
!pip install traitsui
!pip install nipype[all]
!pip install seaborn 

heloo
You should consider upgrading via the '/mnt/new_jxvs2_02/AshleyWorkspace/myenv/bin/python3 -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil
You should consider upgrading via the '/mnt/new_jxvs2_02/AshleyWorkspace/myenv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/mnt/new_jxvs2_02/AshleyWorkspace/myenv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/mnt/new_jxvs2_02/AshleyWorkspace/myenv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/mnt/new_jxvs2_02/AshleyWorkspace/myenv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/mnt/new_jxvs2_02/AshleyWorkspace/myenv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/mnt/new_jxvs2_02/AshleyWorkspace/

In [20]:
#function MARSS_avgMBcorr


In [21]:
#function MARSS_estimateSliceArtifact


In [22]:
#function MARSS_getMPs:  used directly in MARSS
import os
import numpy as np
import nibabel as nib
import shutil
import nipype.interfaces.spm as spm
from nipype.interfaces import fsl
#import nipype as spm #does this work instead of above

def MARSS_getMPs(fn, MB, workingDir):
    # Load volume timeseries
    V = nib.load(fn)

    if V.shape[2] % MB != 0:
        raise ValueError(f"Number of slices and MB factor are incompatible for {fn}.")

    p, f = os.path.split(fn)
    f = os.path.splitext(f)[0]

    # Check workingDir for MPs
    mp_path = os.path.join(workingDir, f'rp_{f}.txt')
    if not os.path.exists(mp_path):
        
        mcflt = fsl.MCFLIRT()
        
        mcflt.inputs.in_file = fn
        
        mcflt.inputs.cost = 'mutualinfo'
        
        rp_path = os.path.join(workingDir, "rp_" + f + ".nii")
        mcflt.inputs.out_file = rp_path

        mcflt.inputs.save_rms = True
        mcflt.inputs.save_plots = True
        # mcflt.inputs.args = "-verbose -1"
        res = mcflt.run()
        
        # Prepare spm job batch
        # a = np.array([fn])
        # matlabbatch = [{'spm.spatial.realign.estimate.data': [a.tolist()],
        #                 'spm.spatial.realign.estimate.eoptions.quality': 0.9000,
        #                 'spm.spatial.realign.estimate.eoptions.sep': 4,
        #                 'spm.spatial.realign.estimate.eoptions.fwhm': 5,
        #                 'spm.spatial.realign.estimate.eoptions.rtm': 1,
        #                 'spm.spatial.realign.estimate.eoptions.interp': 2,
        #                 'spm.spatial.realign.estimate.eoptions.wrap': [0, 0, 0],
        #                 'spm.spatial.realign.estimate.eoptions.weight': ''}]

        # # Run SPM job
        # # spm_jobman.run(matlabbatch)
        # # motion parameters 
        # realign = spm.Realign()
        # # a = spm5_image_list(length(V),{fn}); equivalent?
        # realign.inputs.in_files = fn # data 'functional.nii'
        # realign.inputs.quality = 0.9000
        # realign.inputs.separation = 4
        # realign.inputs.fwhm = 5
        # realign.inputs.register_to_mean = True
        # realign.inputs.interp = 2
        # realign.inputs.wrap = [0, 0, 0]
        # realign.inputs.weight_img = '' #filename of weighting image
        # realign.run()         

        # # Delete .mat file output by SPM
        # if not p:
        #     os.remove(f + '.mat')
        # else:
        #     os.remove(os.path.join(p, f + '.mat'))

    # Copy the MP text file to the working directory
    if not os.path.exists(mp_path):
        mp_file_src = os.path.join(p, f'rp_{f}.txt')
        mp_file_dst = os.path.join(workingDir, f'rp_{f}.txt')
        shutil.copyfile(mp_file_src, mp_file_dst)

    return mp_path


In [23]:
#function MARSS_mbCorrPlot:  used directly in MARSS
def MARSS_mbCorrPlot(timeseriesFile, motionParams, MB):
    # Implementation for MARSS_mbCorrPlot
    pass

In [24]:
#function MARSS_removeSliceArtifact
print("hello")

hello


In [25]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
from scipy.ndimage import rotate
import nibabel as nib

def zToR(z):
    return np.arctanh(z)

def MARSS(timeseriesFile, MB, workingDir):
    # Get name of run
    runName = os.path.splitext(os.path.basename(timeseriesFile))[0]

    # Create a new folder
    runDir = os.path.join(workingDir, runName)
    if not os.path.exists(runDir):
        os.makedirs(runDir)

    # Empty dictionary for all data to go to
    runStruct = {}

    print('Generating pre-MARSS Motion Parameters...')
    # Generate motion parameters for that run and save them
    preMARSS_MPpath = MARSS_getMPs(timeseriesFile, MB, runDir)
    
    # Load motion parameter file and save in DataFrame
    motionParams = pd.read_csv(preMARSS_MPpath)
    runStruct['preMARSS_motionParams'] = motionParams

    print('Generating pre-MARSS Slice Correlations...')
    # Generate slice correlation matrix and save in DataFrame
    preMARSS_corrs = MARSS_mbCorrPlot(timeseriesFile, motionParams, MB)
    preMARSS_sliceCorrelations = {
        'Z_avgSimulSliceCorr_motionRegressed': preMARSS_corrs['rdasimulSlicesZ'],
        'Z_avgAdjacentSliceCorr_motionRegressed': preMARSS_corrs['rdaAdjacentSlicesZ'],
        'R_avgSliceCorrDifference': zToR(preMARSS_corrs['rdasimulSlicesZ'] - preMARSS_corrs['rdaAdjacentSlicesZ']),
        'corrMat_motionRegressed': preMARSS_corrs['corrMat_motionRegressed']
    }
    runStruct['preMARSS_sliceCorrelations'] = pd.DataFrame(preMARSS_sliceCorrelations)
    
    print('Performing MARSS Procedure...')
    # Estimate and remove artifact function
    postMARSS_fname, postMARSS_avgSlcArt_fname = MARSS_removeSliceArtifact(timeseriesFile, MB, motionParams, runDir)
    
    print('Generating post-MARSS Motion Parameters...')
    # Use function to generate motion parameters for new artifact-removed data
    postMARSS_MPpath = MARSS_getMPs(postMARSS_fname, MB, runDir)
    
    # Load new motion parameters and save in DataFrame
    postmotionParams = pd.read_csv(postMARSS_MPpath)
    runStruct['postMARSS_motionParams'] = postmotionParams
    
    print('Generating post-MARSS Slice Correlations...')
    # Generate slice correlation matrix for artifact-removed run and save in DataFrame
    postMARSS_corrs = MARSS_mbCorrPlot(postMARSS_fname, postmotionParams, MB)
    postMARSS_sliceCorrelations = {
        'Z_avgSimulSliceCorr_motionRegressed': postMARSS_corrs['rdasimulSlicesZ'],
        'Z_avgAdjacentSliceCorr_motionRegressed': postMARSS_corrs['rdaAdjacentSlicesZ'],
        'R_avgSliceCorrDifference': zToR(postMARSS_corrs['rdasimulSlicesZ'] - postMARSS_corrs['rdaAdjacentSlicesZ']),
        'corrMat_motionRegressed': postMARSS_corrs['corrMat_motionRegressed']
    }
    runStruct['postMARSS_sliceCorrelations'] = pd.DataFrame(postMARSS_sliceCorrelations)

    # Save final struct 
    sio.savemat(os.path.join(runDir, runName + '_MARSS_SliceCorrelations.mat'), {'runStruct': runStruct})

    # Load avg artifact distribution
    artDistImg = nib.load(postMARSS_avgSlcArt_fname)
    artDistDat = artDistImg.get_fdata()
    maxArt = np.max(artDistDat)
    stdDat = np.std(artDistDat)
    meanArt = np.mean(artDistDat)
    # Color limit for displaying artifact distribution (mean + 3*stddev to control for outlier values)
    colorMax = meanArt + (3 * stdDat)

    # Generate multipanel summary figure
    fig, axs = plt.subplots(2, 6, figsize=(20, 10))

    # Pre-MARSS slice correlation matrix
    axs[0, 0].imshow(runStruct['preMARSS_sliceCorrelations']['corrMat_motionRegressed'], cmap='bwr', vmin=-1, vmax=1)
    axs[0, 0].set_title('Uncorrected Data')
    axs[0, 0].set_xlabel('Slice #')
    axs[0, 0].set_ylabel('Slice #')
    axs[0, 0].axis('equal')
    axs[0, 0].set_xticks([])
    axs[0, 0].set_yticks([])

    # Post-MARSS slice correlation matrix
    axs[0, 1].imshow(runStruct['postMARSS_sliceCorrelations']['corrMat_motionRegressed'], cmap='bwr', vmin=-1, vmax=1)
    axs[0, 1].set_title('Corrected Data')
    axs[0, 1].set_xlabel('Slice #')
    axs[0, 1].set_ylabel('Slice #')
    axs[0, 1].axis('equal')
    axs[0, 1].set_xticks([])
    axs[0, 1].set_yticks([])

    # Orthviews of artifact distribution
    axs[0, 2].imshow(rotate(artDistDat[:, :, artDistDat.shape[2]//2], -90), cmap='hot', vmin=0, vmax=colorMax)
    axs[0, 2].axis('off')

    axs[1, 0].imshow(rotate(artDistDat[:, artDistDat.shape[1]//2, :], 180), cmap='hot', vmin=0, vmax=colorMax)
    axs[1, 0].axis('off')

    axs[1, 1].imshow(rotate(artDistDat[artDistDat.shape[0]//2, :, :], 180), cmap='hot', vmin=0, vmax=colorMax)
    axs[1, 1].axis('off')

    plt.subplots_adjust(wspace=0, hspace=0.2)
    plt.savefig(os.path.join(runDir, 'MARSS_' + runName + '.png'))
    plt.close(fig)

    print('Completed MARSS for ' + runName + '.')


In [ ]:
MARSS(r"/mnt/new_jxvs2_02/AshleyWorkspace/50017_fMRI_Resting_1_AP.nii", 6, r"/mnt/new_jxvs2_02/AshleyWorkspace/50017MARSS/")

Generating pre-MARSS Motion Parameters...


In [ ]:
# !pip install nipype

In [ ]:
import traits


In [ ]:
# import nipype as spm
# pip show nipype

import sys
print(sys.path)
import nipype
print(sys.path)

In [ ]:
import numpy as np

def text_to_matrix(file_path):
    """Converts a text file to a numpy matrix."""

    with open(file_path, 'r') as f:
        lines = f.readlines()

    matrix = []
    for line in lines:
        row = [float(x) for x in line.split()]  # Adjust split() if delimiter is different
        matrix.append(row)

    return np.array(matrix)

matrix = text_to_matrix('/mnt/new_jxvs2_02/AshleyWorkspace/50017MARSS/50017_fMRI_Resting_1_AP/rp_50017_fMRI_Resting_1_AP.nii.par')
print(matrix)

In [ ]:
import numpy as np
import nibabel as nib
timeSeriesDat = nib.load(r"/mnt/new_jxvs2_02/AshleyWorkspace/50017_fMRI_Resting_1_AP.nii");
timeSeriesDat = timeSeriesDat.get_fdata()
# d = squeeze(mean(mean(nY,1),2))';
d = np.mean(np.mean(timeSeriesDat, axis = 0), axis = 0)
d = d.T
print(d)

In [ ]:
# MARSS_mbCorrPlot.m
import matplotlib.pyplot as plt
import seaborn as sns

# % nuisance regressor design matrix [intercept, linearDetrend, MPs,squared
# % MPs, derivatives of MPs, squared derivatives]
# X = [ones(size(d,1),1) (1:size(d,1))' MPs MPs.^2 [zeros(1,6); diff(MPs)] [zeros(1,6); diff(MPs)].^2];
intercept = (np.ones((int(d.shape[0]),1)))
linearDetrend = (np.arange(1, (int(d.shape[0]))+1))
linearDetrend = linearDetrend[:,np.newaxis]
MPs_derivatives = np.vstack(((np.zeros(6), np.diff(matrix,axis = 0))))
squared_MPs_derivatives =  np.vstack((np.zeros(6), np.diff(matrix,axis = 0)))**2 #cannot assign to function call
#print(intercept.shape)
#print(linearDetrend.shape)
#print(MPs_derivatives.shape)
#print(squared_MPs_derivatives.shape)
#print(matrix.shape)
X = np.hstack((intercept, linearDetrend,  matrix, matrix**2, MPs_derivatives, squared_MPs_derivatives))

# rd = zeros(size(d));
rd = np.zeros(d.shape)

# for i = 1:size(d,2)
#     rd(:,i) = d(:,i) - (X*(X\d(:,i)));
# end
for i in range(d.shape[1]):
    # Solve the least-squares problem X \ d[:,i] -> in Python it's np.linalg.lstsq
    beta = np.linalg.lstsq(X, d[:, i], rcond=None)[0]
    
    # Calculate the residual: d[:, i] - X @ beta
    rd[:, i] = d[:, i] - X @ beta

# print(rd)

# out.corrMat_motionRegressed = corr(rd);
corr = np.corrcoef(rd, rowvar=False) 

# Create the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=False, cmap='viridis')
plt.title("Correlation Matrix Heatmap")
plt.show()

In [ ]:
import os
import pandas as pd

os.getcwd()
print(corr)
# np.savetxt("foo.csv", corr, delimiter=",")
my_df = pd.DataFrame(corr)
my_df.to_csv('/mnt/new_jxvs2_02/AshleyWorkspace/foo.csv', index=False)

In [ ]:
import csv
# MARSS_avgMBcorr.m
def MARSS_avgMBcorr(dat, MB):
    c = np.corrcoef(dat, rowvar=False)
    cZ = 0.5 * np.log((1 + c) / (1 - c))
    sgap = c.shape[0] / MB

    # [avgS_Z, avgNS_Z] = deal(zeros(size(c, 1), 1));
    num_rows = c.shape[0]  
    avgS_Z = np.zeros((num_rows, 1))
    avgNS_Z = np.zeros((num_rows, 1))

    # empty array to append ind to 
    rows = []

    for i in range(c.shape[0]):
        cntS = 0;
        cntNS = 0; #where is this used?
        # from i-sgap to 1 in increments of -sgap
        # ind = [(i-sgap):-sgap:1 (i+sgap):sgap:size(c,1)];
        # ind = np.concatenate((np.arange(i - sgap, 0, -sgap), np.arange(i + sgap, num_rows+1, sgap)))
        ind = np.concatenate((np.arange(i - sgap, -1, -sgap), np.arange(i + sgap, num_rows, sgap)))     
        ind = np.array(ind, dtype=int)
        
        # # avgS_Z(i) = mean(cZ(i,ind));
        # avgS_Z[i] = np.mean(cZ[i, ind-1])
        avgS_Z[i] = np.mean(cZ[i, ind])
        
        #adjacent slices
        # ind = [ind-1 ind+1];
        # ind(ind<1|ind>size(c,1)) = [];
        # avgNS_Z(i) = mean(cZ(i,ind));
        ind = np.concatenate((ind - 1, ind + 1))  # Create adjacent indices
        ind = ind[(ind >= 0) & (ind <= num_rows)]  # Keep only valid indices
        # WARNING: THESE CORRELATIONS ARE VERY DIFFERENT THAN THE MATLAB VERSION OF MARSS AS OF NOW
        avgNS_Z[i] = np.mean(cZ[i, ind-1])  # Adjust index for zero-based indexing
    print(np.mean(avgS_Z - avgNS_Z))

    # simulSlicesZ = np.mean(avgS_Z)
    # adjacentSlicesZ = np.mean(avgNS_Z)

    # print(simulSlicesZ)
    # print(adjacentSlicesZ)

 #   with open('indices.csv', mode='w', newline='') as file:
 #       writer = csv.writer(file)
 #       writer.writerows(rows)
    
MARSS_avgMBcorr(rd,6)

In [36]:
# MARSS_estimateSliceArtifact.m
from scipy.stats import zscore

def MARSS_estimateSliceArtifact(Y,MB,X):

    # if mod(size(Y,3),MB)
    #     error('# of slices and MB factor are incompatible.');
    # end
    if Y.shape[2] % MB != 0:
        raise ValueError("# of slices and MB factor are incompatible.")
    
    # nSlices = size(Y,3);
    # nSliceSets = nSlices/MB;
    # data = squeeze(mean(mean(Y,1),2))';
    nSlices = Y.shape[2]
    nSliceSets = nSlices // MB
    data = np.squeeze(np.mean(np.mean(Y, axis=1), axis=0)).T
    # print(nSlices, nSliceSets, data)

    # X = zscore(X);
    # X = zscore(X, axis=0, ddof=1) #-0.84671618 -0.82574562  1.40956156
    X = zscore(X, axis=0, ddof=1) #-0.84746915 -0.82647994  1.41081506
    #print(X)

    # [sameSetData,artifactEstimate,globalSignal] = deal(zeros(size(data)));
    sameSetData = np.zeros_like(data)
    artifactEstimate = np.zeros_like(data)
    globalSignal = np.zeros_like(data)

    for j in range(nSlices):
        sliceSet = j % nSliceSets
        if sliceSet == 0:
            sliceSet = nSliceSets
        
        sameSliceSet = np.zeros(nSlices, dtype=bool)  # Initialize boolean array for same slice set
        lowerSlices = np.arange(sliceSet, j, nSliceSets)  # Get lower slices
        higherSlices = np.arange(j + nSliceSets, nSlices, nSliceSets)  # Get higher slices
        sameSliceSet[np.concatenate((lowerSlices, higherSlices))] = True  # Set slices in the same slice set
    
        # Calculate sameSetData for the current slice j
        sameSetData[:, j] = np.mean(data[:, sameSliceSet], axis=1)
    
        # Exclude current slice from global signal calculation
        sameSliceSet[j] = True
        globalSignal[:, j] = np.mean(data[:, ~sameSliceSet], axis=1)
    
        # Design matrix: [intercept, globalSignal, MPs (X)]
        Xout = np.column_stack((np.ones(data.shape[0]), zscore(globalSignal[:, j],ddof = 1), X))
    
        # Perform the regression (solve the system of equations)
        B = np.linalg.lstsq(Xout, sameSetData[:, j], rcond=None)[0]
    
        # Compute artifact estimate
        artifactEstimate[:, j] = sameSetData[:, j] - Xout @ B
    return [artifactEstimate, globalSignal]
    # return artifcatEstimate

    # print(globalSignal)
    # print(Xout)
    # print(B)
    # print(artifactEstimate[0,:])


In [ ]:
# Xest = [MPs MPs.^2 [zeros(1,6); diff(MPs)] [zeros(1,6); diff(MPs)].^2];
Xest = np.hstack((matrix, matrix**2, MPs_derivatives, squared_MPs_derivatives))

MARSS_estimateSliceArtifact(timeSeriesDat,6,Xest)

In [65]:
import numpy as np
import nibabel as nib
import os

# MARSS_removeSliceArtifact.m
def MARSS_removeSliceArtifact(filename,MB,MPs,workingDir):
    # V = spm_vol(filename{i});
    # Y = spm_read_vols(V);
    # for j = 1:length(V)
    #     V(j).dt = [16 0];
    # end
    # Y = double(Y);         
    img = nib.load(filename)
    Y = img.get_fdata()
    V = img.header
    # Modify the header data type (equivalent to V(j).dt = [16 0])
    # In Nibabel, you modify the datatype using the header object
    hdr = img.header
    hdr.set_data_dtype(np.float32)  # Equivalent to SPM's [16 0] (single precision float)
    # Convert the data to double precision (equivalent to Y = double(Y))
    Y = Y.astype(np.float64)

    # if mod(size(Y,3),MB)
    #     warning(['# of slices and MB factor are incompatible for ' filename{i} '. Skipping.']);
    #     continue
    # end    
    if Y.shape[2] % MB != 0:
        print(f"Warning: # of slices and MB factor are incompatible for {filename}. Skipping.")
        

    MPs = (MPs - np.mean(MPs)) / np.std(MPs)  # Z-score normalization
    # Nuisance parameter design matrix
    Xest = np.hstack([
            MPs,
            MPs ** 2,
            np.vstack([np.zeros((1, 6)), np.diff(MPs, axis=0)]),
            np.vstack([np.zeros((1, 6)), np.diff(MPs, axis=0)]) ** 2
    ])
    
    # Estimate artifact signal in each slice
    [artifactEstimate, nonsliceGlobalSignal] = MARSS_estimateSliceArtifact(Y, MB, Xest)     
    # artifactEstimate = MARSS_estimateSliceArtifact(Y, MB, Xest)     
    print(artifactEstimate) #doesnt match
    print(nonsliceGlobalSignal) #matches


    Ya = np.zeros_like(Y)
    Yart = np.zeros_like(Y)   
    for j in range(artifactEstimate.shape[1]):
        # Final MARSS design matrix
        Xcalc = np.column_stack([np.ones(artifactEstimate.shape[0]), (artifactEstimate[:, j] - np.mean(artifactEstimate[:, j])) / np.std(artifactEstimate[:, j]), 
                                 (nonsliceGlobalSignal[:, j] - np.mean(nonsliceGlobalSignal[:, j])) / np.std(nonsliceGlobalSignal[:, j]),
                                 (Xest - np.mean(Xest, axis=0)) / np.std(Xest, axis=0)])

        Yt = Y[:, :, j, :].reshape(-1, Y.shape[3]).T
        # Perform regression
        B = np.linalg.lstsq(Xcalc, Yt, rcond=None)[0]

        print(Xcalc.shape)
        print(B.shape)

        # art = Xcalc[:, 1] @ B[1, :]
        art = np.dot(Xcalc[:,1], B[1,:])
        # Subtract artifact estimation from original timeseries data
        # Yta = (Yt.T - art.T).T.reshape(Y.shape[0], Y.shape[1], 1, Y.shape[3])
        # Yartt = art.reshape(Y.shape[0], Y.shape[1], 1, Y.shape[3])
        Y_diff = Yt.T - art.T
        Yta = np.reshape(Y_diff, (Y.shape[0], Y.shape[1], 1, Y.shape[3]))

        Yartt = np.reshape(art.T, (Y.shape[0], Y.shape[1], 1, Y.shape[3]))
        Ya[:, :, j, :] = Yta
        Yart[:, :, j, :] = Yartt

        # Create filenames for output
        base_name = os.path.basename(filename)
        f, x = os.path.splitext(base_name)

    
    for j in range(Y.shape[3]):
        # Corrected Data
        corrected_filename = os.path.join(working_dir, f'za{f}{x}')
        nib.save(nib.Nifti1Image(Ya[:, :, :, j], V.affine), corrected_filename)

        # Isolated artifact timeseries
        artifact_filename = os.path.join(working_dir, f'{f}_slcart{x}')
        nib.save(nib.Nifti1Image(Yart[:, :, :, j], V.affine), artifact_filename)

    Yaavg = np.mean(np.abs(Yart), axis=3)

    # Average artifact distribution
    avg_artifact_filename = os.path.join(working_dir, f'{f}_AVGslcart{x}')
    nib.save(nib.Nifti1Image(Yaavg, V.affine), avg_artifact_filename)

    postMARSS_fname = os.path.join(working_dir, f'za{f}{x}')
    postMARSS_avgSlcArt_fname = os.path.join(working_dir, f'{f}_AVGslcart{x}')    
        
    return [postMARSS_fname, postMARSS_avgSlcArt_fname]
    

In [66]:
import numpy as np

def text_to_matrix(file_path):
    """Converts a text file to a numpy matrix."""

    with open(file_path, 'r') as f:
        lines = f.readlines()

    matrix = []
    for line in lines:
        row = [float(x) for x in line.split()]  # Adjust split() if delimiter is different
        matrix.append(row)

    return np.array(matrix)

matrix = text_to_matrix('/mnt/new_jxvs2_02/AshleyWorkspace/50017MARSS/50017_fMRI_Resting_1_AP/rp_50017_fMRI_Resting_1_AP.nii.par')
# print(matrix)

# print(MARSS_removeSliceArtifact(r"/mnt/new_jxvs2_02/AshleyWorkspace/50017_fMRI_Resting_1_AP.nii", 6, matrix, r"/mnt/new_jxvs2_02/AshleyWorkspace/50017MARSS/"))
MARSS_removeSliceArtifact(r"/mnt/new_jxvs2_02/AshleyWorkspace/50017_fMRI_Resting_1_AP.nii", 6, matrix, r"/mnt/new_jxvs2_02/AshleyWorkspace/50017MARSS/")

[[ -1.71494632   2.13758072  11.81613784 ...  -7.40061324 -22.21833089
  -11.93248192]
 [  6.54115167   6.15447477   1.97877252 ... -15.44939275  -8.9681222
   -8.41661237]
 [ -1.14811289   5.37418075  13.21684273 ... -16.43093953  -9.69274102
   -2.30996263]
 ...
 [  1.75718448   4.38800212 -16.15310447 ...  10.58876624  12.85481771
   11.78239785]
 [ 21.69182573  20.06201374  24.70880415 ...  -3.4230664  -10.93473689
   -6.47848418]
 [  4.7996439    1.82354335  -0.25277928 ...   9.57120841  -5.77576244
  -10.67072586]]
[[3777.26281558 3775.76363098 3776.20710945 ... 3794.48939991
  3799.03660771 3800.22428874]
 [3753.99259419 3753.77260829 3756.11083237 ... 3773.71527938
  3776.62852589 3778.61774318]
 [3746.72827278 3745.41545079 3746.3210512  ... 3766.07977861
  3768.8424308  3770.21426535]
 ...
 [3745.21166378 3745.28767141 3747.88859733 ... 3764.74356658
  3767.84000384 3768.87182013]
 [3740.63444669 3740.52653467 3740.9705594  ... 3761.96204825
  3765.4336361  3766.15278739]
 [3

ValueError: could not broadcast input array from shape (102,102,1,563) into shape (102,102,563)

here
